In [124]:
import numpy as np

In [190]:
tmp = np.load("../test_messung_burstCnt_1.npz", allow_pickle=True)
tmp.files

['data_hex', 'data_int']

In [191]:
def conv_hex_list(lst: list):
    data = [ele.replace("0x", "") for ele in lst]

    data_new = []
    for ele in data:
        if len(ele) == 1:
            apdx = "0"
            data_new.append(apdx + ele)
        else:
            apdx = "0x"
            data_new.append(ele)
    data_new = [bytes.fromhex(ele) for ele in data_new]
    int_data = [int.from_bytes(ele, 'big') for ele in data_new]
    return data_new, int_data

In [192]:
dt_hex, dt_int = conv_hex_list(tmp["data_hex"])

In [200]:
dt_hex[len(dt_hex)-20:]

[b'\xf7',
 b']',
 b'\xde',
 b'\xb5',
 b'\xff',
 b'\xaf',
 b'\x95',
 b'\xb5',
 b'\xf7',
 b']',
 b'\xde',
 b'\xb5',
 b'\xff',
 b'\xaf',
 b'\x95',
 b'\xb5',
 b'\xf7',
 b']',
 b'\xde',
 b'\xb4']

In [198]:
len(dt_hex)-10

17914

In [199]:
tmp["data_hex"][17914:]

array(['0xde', '0xb5', '0xff', '0xaf', '0x95', '0xb5', '0xf7', '0x5d',
       '0xde', '0xb4'], dtype='<U4')